In [5]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# check os platform
import platform
print('OS: {}'.format(platform.platform()))

1.7.0 True
0.14.0
11.2
GCC 7.5
OS: Linux-5.4.0-84-generic-x86_64-with-glibc2.10


### CONFIG MMACTION2
1. Option (RECOMMENDED): create a customconfig.py file 
2. Option: modify other config and train from script (see folder MMACTION2/TESTS/mmaction2_tutorial)

In [6]:
# mmaction2 dir
mmadir="C:/Users/nmc_costa/Desktop/Desk/mmaction2/" #"/home/administrator/Z/Algorithms/mmaction2/"
configdir= "C:/Users/nmc_costa/google_drive/projects/bosch_P19/research/python_ws/violent_action/AR/image_based/MMACTION2/CONFIGS/"#"/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/CONFIGS/"
work_dir_root= mmadir
if str(platform.platform()).upper().find('linux'.upper())>-1: 
    mmadir="/home/administrator/Z/Algorithms/mmaction2/"
    configdir="/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/CONFIGS/"
    work_dir_root= '/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/TESTS/train/' #defualt: './'
print(mmadir)
print(configdir)
print(work_dir_root)

from mmcv import Config

root= mmadir+"data/" #default: "data/"
dataset="INCAR"
dataset_type = 'RawframeDataset'
config_file='tsn_r50_1x1x3_100e_INCAR_rgb.py'
if dataset_type=='RawframeDataset': 
    dtype_name="rawframes"
    cfg = Config.fromfile(mmadir+'/configs/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb.py') #Config.fromfile(configdir+config_file) 
if dataset_type=='VideoDataset': 
    dtype_name="videos"
    config_file='tsn_video_r50_1x1x8_100e_INCAR_rgb.py'
    cfg = Config.fromfile(configdir+'tsn_video_r50_1x1x8_100e_INCAR_rgb.py') 

/home/administrator/Z/Algorithms/mmaction2/
/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/CONFIGS/
/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/TESTS/train/


In [7]:
filename_tmpl='img_{:05}.png' #filename template of saved frames

# Modify dataset type and path
cfg.dataset_type = dataset_type
cfg.data_root =  root+dataset+'/'+dtype_name
cfg.data_root_val = cfg.data_root
cfg.ann_file_train =  root+dataset+'/'+dataset+'_train_'+dtype_name+'.txt'
cfg.ann_file_val = root+dataset+'/'+dataset+'_val_'+dtype_name+'.txt'
cfg.ann_file_test = root+dataset+'/'+dataset+'_test_'+dtype_name+'.txt'

cfg.data.train.type = dataset_type
cfg.data.train.ann_file = cfg.ann_file_train
cfg.data.train.data_prefix = cfg.data_root
cfg.data.train.filename_tmpl=filename_tmpl #filename template of saved frames

cfg.data.val.type = dataset_type
cfg.data.val.ann_file = cfg.ann_file_val
cfg.data.val.data_prefix = cfg.data_root
cfg.data.val.filename_tmpl=filename_tmpl #filename template of saved frames

cfg.data.test.type = dataset_type
cfg.data.test.ann_file = cfg.ann_file_test
cfg.data.test.data_prefix = cfg.data_root 
cfg.data.test.filename_tmpl=filename_tmpl #filename template of saved frames

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = None#'https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmaction/mmaction-v1/recognition/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = work_dir_root+'work_dirs/'+config_file.split('.')[0]+'/'#'work_dirs/tsn_kinetics_pretrained_r50_1x1x3_50e_INCAR_rgb/'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
# we also only use 2 videos per gpu
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 2

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
from mmcv.runner import set_random_seed
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01),
    train_cfg=None,
    test_cfg=dict(average_clips=None))
optimizer = dict(type='SGD', lr=7.8125e-05, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=40, norm_type=2))
lr_config = dict(policy='step', step=[40, 80])
total_epochs = 2
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
dataset_type = 'RawframeDataset'
data_root = '/home/administrator/Z/Algorithms/mmaction2/data/INCAR/rawframes'

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [8]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True) 

2021-09-22 18:28:58,918 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.bias', 'fc.weight'}


Use load_from_torchvision loader


2021-09-22 18:29:00,813 - mmaction - INFO - Start running, host: administrator@administrator, work_dir: /home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/TESTS/train/work_dirs/tsn_r50_1x1x3_100e_INCAR_rgb
2021-09-22 18:29:00,815 - mmaction - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(A

OSError: [Errno 95] Operation not supported: 'epoch_2.pth' -> '/home/administrator/Z/Work/EASYRIDE/P19/NC/mmaction2/TESTS/train/work_dirs/tsn_r50_1x1x3_100e_INCAR_rgb/latest.pth'

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [9]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 391/391, 1.3 task/s, elapsed: 307s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	0.9003
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	0.8971
top1_acc: 0.9003
top5_acc: 1.0000
mean_class_accuracy: 0.8971
